# A first stab: DQN

[DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) is a classical RL algorithm which should provide a nice baseline for further work.

Classical RL techniques woul probably not work very well without further feature engineering, because the current state space is quite large.

In [1]:
import tianshou as ts 
from tianshou.utils import TensorboardLogger

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

import numpy as np

import os
from datetime import datetime

In [2]:
from utils_preprocess import compute_frame_features, compute_foa_features

from env_base import BaseEnvironment
from env_base_test import BaseTestEnvironment

/Users/johndoe/Desktop/uni/affectivenatural/project/lib/python3.11/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists


## Data and environment initialisation

In [3]:
vid_filename = "001"
mat_filename = vid_filename + ".mat"
target_subject = 0

In [4]:
patch_bounding_boxes_per_frame, patch_centres_per_frame, speaker_info_per_frame = compute_frame_features(
    vid_filename
)

foa_centres_per_frame_per_subject, patch_weights_per_frame = compute_foa_features(
    mat_filename, patch_centres_per_frame
)
foa_centres_per_frame = [frame[target_subject] for frame in foa_centres_per_frame_per_subject]

In [5]:
markov_env = BaseEnvironment(
    1,
    patch_bounding_boxes_per_frame,
    patch_centres_per_frame,
    speaker_info_per_frame,
    foa_centres_per_frame,
    patch_weights_per_frame,
    frame_width=320, # from data_utils.py
    frame_height=180,
)

In [6]:
# env.observation_space.sample(), env.action_space.sample()

For efficiency, it's a good idea to set up some vectorized environments.

In [7]:
num_train_envs = 5
num_test_envs = 10

train_envs = ts.env.DummyVectorEnv([lambda: markov_env for _ in range(num_train_envs)])
test_envs = ts.env.DummyVectorEnv([lambda: markov_env for _ in range(num_test_envs)])

## DQN

First, let's construct the network.

The biggest headache comes from the observations: they're quite complex. So, we build multiple networks, each processing a part of an observation and combining their outputs in the end!

In [8]:
class Net(nn.Module):
    def __init__(self, observation_space, action_shape):
        super().__init__()

        self.num_patches = observation_space['patch_centres'].shape[0]

        # network for patch_centres
        self.patch_centres_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['patch_centres'].shape), 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True)
        )

        # network for patch_bounding_boxes
        self.patch_bboxes_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['patch_bounding_boxes'].shape), 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True)
        )

        # network for speaker_info
        self.speaker_info_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['speaker_info'].shape), 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 32),
            nn.ReLU(inplace=True)
        )

        # combining the outputs of all networks
        self.combined_net = nn.Sequential(
            nn.Linear(64 + 64 + 32, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape))
        )

    def forward(self, obs, state=None, info={}):
        patch_centres = torch.tensor(obs['patch_centres'], dtype=torch.float32)
        patch_bboxes = torch.tensor(obs['patch_bounding_boxes'], dtype=torch.float32)
        speaker_info = torch.tensor(obs['speaker_info'], dtype=torch.float32)

        patch_centres = patch_centres.view(patch_centres.size(0), -1)
        patch_bboxes = patch_bboxes.view(patch_bboxes.size(0), -1)
        speaker_info = speaker_info.view(speaker_info.size(0), -1)

        # pass through respective networks
        patch_centres_out = self.patch_centres_net(patch_centres)
        patch_bboxes_out = self.patch_bboxes_net(patch_bboxes)
        speaker_info_out = self.speaker_info_net(speaker_info)

        # combine outputs
        combined = torch.cat([patch_centres_out, patch_bboxes_out, speaker_info_out], dim=1)

        logits = self.combined_net(combined)

        return logits, state

In [9]:
state_shape = markov_env.observation_space
action_shape = markov_env.action_space.n

net = Net(state_shape, action_shape)
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

In [10]:
net

Net(
  (patch_centres_net): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU(inplace=True)
  )
  (patch_bboxes_net): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU(inplace=True)
  )
  (speaker_info_net): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU(inplace=True)
  )
  (combined_net): Sequential(
    (0): Linear(in_features=160, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

## Setting up DQN

First, we need to set up the policy, which is readily done in Tianshou.

In [11]:
policy = ts.policy.DQNPolicy(
    model=net, 
    optim=optim, 
    discount_factor=0.99,
    estimation_step=1,
    target_update_freq=50
)

Then, we need to set up the collectors, i.e., the objects that will be interacting with the environment according to the above policy and collect the generated data.

In classical DQN fashion, we store the data in a replay buffer.

In [12]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(6000, num_train_envs))

test_collector = ts.data.Collector(policy, test_envs)

## Training

In [13]:
num_epochs = 10
num_steps_per_epoch = 300
step_per_collect = 10
episode_per_test = 5
batch_size = 30 # one second of data (videos are at 30FPS)

timestamp = datetime.now().strftime("%d%m%Y-%H%M%S")
log_path = os.path.join("logs", "dqn", "base", f"video_{vid_filename}", f"subject_{target_subject}", timestamp)
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)

In [14]:
result = ts.trainer.offpolicy_trainer(
    policy, 
    train_collector, 
    test_collector,
    max_epoch=num_epochs,
    step_per_epoch=num_steps_per_epoch,
    step_per_collect=step_per_collect,
    episode_per_test=episode_per_test,
    batch_size=batch_size,
    logger=logger,
)

Epoch #1: 301it [00:00, 454.24it/s, env_step=300, len=0, loss=0.757, n/ep=0, n/st=10, rew=0.00]                         


Epoch #1: test_reward: 413.805613 ± 344.469630, best_reward: 413.805613 ± 344.469630 in #1


Epoch #2: 301it [00:00, 501.56it/s, env_step=600, len=0, loss=0.954, n/ep=0, n/st=10, rew=0.00]                         


Epoch #2: test_reward: 413.805613 ± 340.183569, best_reward: 413.805613 ± 344.469630 in #1


Epoch #3: 301it [00:00, 495.12it/s, env_step=900, len=0, loss=1.762, n/ep=0, n/st=10, rew=0.00]                         


...

Epoch #8: test_reward: 413.805613 ± 349.556102, best_reward: 413.805613 ± 344.469630 in #1


Epoch #9: 301it [00:00, 486.89it/s, env_step=2700, len=0, loss=1.206, n/ep=0, n/st=10, rew=0.00]                         


Epoch #9: test_reward: 413.805613 ± 333.741414, best_reward: 413.805613 ± 344.469630 in #1


Epoch #10: 301it [00:00, 497.67it/s, env_step=3000, len=0, loss=1.168, n/ep=0, n/st=10, rew=0.00]                         


Epoch #10: test_reward: 413.805613 ± 337.912158, best_reward: 413.805613 ± 344.469630 in #1


Save the weights of the current module (for further testing and improvements down the line).

In [24]:
policy_path = os.path.join("weights", "dqn", "base", f"video_{vid_filename}", f"subject_{target_subject}")
# TODO handle the error in case the path already exists (i.e., just continue)
os.makedirs(policy_path)

torch.save(policy.state_dict(), os.path.join(policy_path, "weights.pth"))

In [25]:
result

{'duration': '8.81s',
 'train_time/model': '5.79s',
 'test_step': 19789,
 'test_episode': 55,
 'test_time': '2.61s',
 'test_speed': '7576.93 step/s',
 'best_reward': 413.8056129167984,
 'best_result': '413.81 ± 344.47',
 'train_step': 3000,
 'train_episode': 0,
 'train_time/collector': '0.40s',
 'train_speed': '484.14 step/s'}

Well, that's quite a let down...

Although, there's not much to be surprised about: there is so very little information passed to the networks! 

Plus, I'm still not too sure that the problem is really amenable to RL in the first place.

At this point, I have two choices:
1. I keep fine-tuning hyperparameters until I get an acceptable result,
2. I try a different approach.

I'll opt for the second option, but I made this code into a notebook precisely because, that way, tinkering would be easier. So, if you wish to tune and fine-tune things, go ahead!

## Testing

In [30]:
# #! make sure you run all the code up to the instantiation of the models and optimizers before this cell
policy = ts.policy.DQNPolicy(
    model=net, 
    optim=optim, 
    discount_factor=0.99,
    estimation_step=1,
    target_update_freq=50
)

policy.load_state_dict(torch.load(os.path.join(policy_path, "weights.pth")))

<All keys matched successfully>

In [31]:
test_markov_env = BaseTestEnvironment(
    1,
    patch_bounding_boxes_per_frame,
    patch_centres_per_frame,
    speaker_info_per_frame,
    foa_centres_per_frame,
    patch_weights_per_frame,
    frame_width=320, # from data_utils.py
    frame_height=180,
)

In [32]:
num_test_envs = 1 # need set it to 1 (else it doesn't get to the end)

testing_envs = ts.env.DummyVectorEnv([lambda: test_markov_env for _ in range(num_test_envs)])

In [33]:
policy.eval()
policy.set_eps(0.05)

collector = ts.data.Collector(policy, testing_envs)
# should be the same values 3 times (if not, there's a problem)
print(collector.collect(n_episode=3))

{'n/ep': 3, 'n/st': 1797, 'rews': array([400., 400., 400.]), 'lens': array([599, 599, 599]), 'idxs': array([0, 0, 0]), 'rew': 400.0, 'len': 599.0, 'rew_std': 0.0, 'len_std': 0.0}


### TensorBoard visualisation

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
%tensorboard --logdir logs/dqn

Reusing TensorBoard on port 6006 (pid 3699), started 0:00:20 ago. (Use '!kill 3699' to kill it.)